# 🤖 Module 2 Video 5: Sequential Chat with Tool Use in AutoGen

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Sequential Chats**: Learn how to chain multiple agent conversations where each chat passes context to the next
2. **Implement Tool Integration**: Master how to give agents access to external tools and APIs
3. **Use LangChain Tool Interoperability**: Convert LangChain tools to work with AutoGen agents
4. **Create Custom Tools**: Build your own functions and register them as tools for agents
5. **Build Multi-Agent Workflows**: Create a complete pipeline that searches, aggregates, and formats information

---

# 📋 Problem Statement

## Business Scenario

You are the **Learning & Development (L&D) Head** at a technology company. Your goal is to:

- 📰 **Create a curated newsletter** for employees on AI Agents and Generative AI
- 📚 **Source content** from Analytics Vidhya's blogs for professional insights
- 🤖 **Automate the process** using an agentic system that:
  - Queries Analytics Vidhya for recent articles
  - Searches for related news using Tavily Search
  - Compiles everything into a professional newsletter email

## The Solution Architecture

We'll build a **Sequential Chat Pipeline** with three specialized agents:

```
┌──────────────────┐    ┌──────────────────┐    ┌──────────────────┐
│   AV Blog Agent  │───▶│    News Agent    │───▶│   Email Agent    │
│  (Web Scraping)  │    │ (Tavily Search)  │    │(Newsletter Gen)  │
└──────────────────┘    └──────────────────┘    └──────────────────┘
        │                       │                        │
        ▼                       ▼                        ▼
   AV Articles              + News                  Newsletter
   with URLs              Results                    Email
```

---

## Step 1: Install Required Dependencies

Before we begin, we need to install the necessary packages:

| Package | Purpose |
|---------|--------|
| `tavily-python` | Search API for retrieving real-time web results |
| `langchain` | Framework for building LLM applications |
| `langchain-community` | Community tools including Tavily integration |

> **Note**: You'll need a Tavily API key. Get one free at [tavily.com](https://tavily.com)

In [ ]:
# ============================================================================
# STEP 1: INSTALL REQUIRED PACKAGES
# ============================================================================
# Install Tavily SDK for web search capabilities
# Install LangChain packages for tool interoperability

%pip install -q tavily-python     # Tavily search API client
%pip install -q langchain         # LangChain core framework
%pip install -q langchain-community  # Community integrations (includes Tavily tool)

---

## Step 2: Import Required Libraries

Let's import all the libraries we'll need:

- **AutoGen components**: For building conversable agents
- **LangChain tools**: For Tavily search integration
- **Interoperability**: To convert LangChain tools to AutoGen format
- **Web scraping libraries**: For extracting content from Analytics Vidhya

In [ ]:
# ============================================================================
# STEP 2: IMPORT REQUIRED LIBRARIES
# ============================================================================

# AutoGen imports for multi-agent systems
import autogen
from autogen import (
    ConversableAgent,    # Base class for all conversable agents
    UserProxyAgent,      # Agent that executes code/tools on behalf of user
    AssistantAgent       # AI-powered assistant agent
)
from autogen import register_function  # For registering custom tools

# LangChain integration for Tavily search
from langchain_community.tools import TavilySearchResults

# AutoGen interoperability layer - converts external tools to AutoGen format
from autogen.interop import Interoperability

# Web scraping libraries for Analytics Vidhya
import requests                        # HTTP requests
from bs4 import BeautifulSoup          # HTML parsing

# Display utilities
from IPython.display import Markdown, display

print("✅ All libraries imported successfully!")

---

## Step 3: Load Environment Variables

We use `.env` files to securely store API keys. Your `.env` file should contain:

```
OPENAI_API_KEY=your_openai_api_key_here
TAVILY_API_KEY=your_tavily_api_key_here
```

> ⚠️ **Security Best Practice**: Never commit API keys to version control!

In [ ]:
# ============================================================================
# STEP 3: LOAD ENVIRONMENT VARIABLES
# ============================================================================
# Load API keys from .env file for secure credential management
# Required keys: OPENAI_API_KEY, TAVILY_API_KEY

from dotenv import load_dotenv

# Load environment variables from .env file in the project root
load_dotenv()

print("✅ Environment variables loaded!")

---

# 🔗 Sequential Chat with Tool Use

## What is a Sequential Chat?

A **Sequential Chat** is a pattern where:
1. Multiple agent conversations happen one after another
2. The output/summary from one chat becomes the input context for the next
3. Each agent specializes in a specific task

## Key Features:

| Feature | Description |
|---------|-------------|
| **Context Passing** | Each chat receives a summary of previous chats |
| **Tool Integration** | Agents can use external tools to complete tasks |
| **Specialization** | Each agent focuses on one specific capability |
| **Chaining** | Complex workflows broken into manageable steps |

---

## Step 4: Configure the LLM

Set up the configuration for our language model. We use `gpt-4o-mini` which provides:
- Fast response times
- Cost-effective processing
- Good quality for tool use tasks

In [ ]:
# ============================================================================
# STEP 4: CONFIGURE THE LANGUAGE MODEL
# ============================================================================
# Define the LLM configuration that will be used by all agents
# Using gpt-4o-mini for cost-effective tool use capabilities

config_list = {
    "config_list": [
        {
            "model": "gpt-4o-mini",  # Cost-effective model with good tool use
            "temperature": 0.1       # Low temperature for consistent outputs
        }
    ],
}

print("✅ LLM configuration ready!")
print(f"   Model: gpt-4o-mini")
print(f"   Temperature: 0.1")

---

## Step 5: Create the Specialized Agents

We'll create four agents:

1. **AV Blog Agent** 📚 - Searches Analytics Vidhya for articles
2. **User Proxy** 👤 - Executes tools on behalf of the user
3. **News Agent** 📰 - Finds related news using Tavily
4. **Email Agent** ✉️ - Writes the final newsletter

### Agent Roles Diagram

```
                        ┌─────────────────┐
                        │   User Proxy    │
                        │ (Tool Executor) │
                        └────────┬────────┘
                                 │
         ┌───────────────────────┼───────────────────────┐
         │                       │                       │
         ▼                       ▼                       ▼
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  AV Blog Agent  │    │   News Agent    │    │   Email Agent   │
│   (Scraping)    │    │    (Search)     │    │  (Formatting)   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

In [ ]:
# ============================================================================
# STEP 5A: CREATE THE AV BLOG AGENT
# ============================================================================
# This agent specializes in searching Analytics Vidhya blogs
# It will use a custom web scraping tool to fetch articles

av_blog_agent = ConversableAgent(
    name="av_blog_agent",
    system_message="""You are a helpful AI assistant specialized in finding 
    content from Analytics Vidhya blogs. You can use the given tool to get 
    the relevant results. When presenting results, include the article title, 
    description, and URL. Return 'TERMINATE' when the task is done.""",
    llm_config=config_list
)

print("✅ AV Blog Agent created!")

In [ ]:
# ============================================================================
# STEP 5B: CREATE THE USER PROXY AGENT
# ============================================================================
# The UserProxyAgent serves as the tool executor
# It doesn't have an LLM - it just executes tools and returns results

user_proxy = UserProxyAgent(
    name="User",
    
    # No LLM for this agent - it just executes tools
    llm_config=False,
    
    # Define when to stop the conversation
    # Stops when it receives a message containing "TERMINATE"
    is_termination_msg=lambda msg: (
        msg.get("content") is not None and "TERMINATE" in msg["content"]
    ),
    
    # "TERMINATE" means ask for human input only when chat terminates
    # Other options: "ALWAYS", "NEVER"
    human_input_mode="TERMINATE",
    
    # Disable code execution (we only want tool execution)
    code_execution_config=False
)

print("✅ User Proxy Agent created!")

In [ ]:
# ============================================================================
# STEP 5C: CREATE THE NEWS AGENT
# ============================================================================
# This agent uses Tavily Search to find related news articles
# It enriches the content with recent news and updates

news_agent = ConversableAgent(
    name="news_agent",
    system_message="""You are a helpful AI assistant specialized in finding 
    the latest news. You can use the given tool to get relevant search results.
    When presenting results, organize them by topic and include URLs.
    Return 'TERMINATE' when the task is done.""",
    llm_config=config_list
)

print("✅ News Agent created!")

---

## Step 6: Set Up the Tavily Search Tool

### What is Tavily?

**Tavily** is a search API optimized for AI agents that provides:
- Real-time web search results
- Content extraction and summarization
- Structured output format

### Why Use Interoperability?

LangChain has many useful tools, but they're not directly compatible with AutoGen.
The `Interoperability` class converts LangChain tools to AutoGen format:

```
┌──────────────────┐    ┌─────────────────────┐    ┌──────────────────┐
│  LangChain Tool  │───▶│  Interoperability   │───▶│   AutoGen Tool   │
│  (TavilySearch)  │    │     .convert_tool() │    │   (ag2_tool)     │
└──────────────────┘    └─────────────────────┘    └──────────────────┘
```

In [ ]:
# ============================================================================
# STEP 6A: CREATE THE TAVILY SEARCH TOOL
# ============================================================================
# Initialize the Tavily search tool from LangChain
# This requires TAVILY_API_KEY to be set in your environment

tavily_tool = TavilySearchResults(
    max_results=5  # Return up to 5 search results per query
)

print("✅ Tavily Search Tool created!")
print(f"   Max results per query: 5")

In [ ]:
# ============================================================================
# STEP 6B: CONVERT LANGCHAIN TOOL TO AUTOGEN FORMAT
# ============================================================================
# Use AutoGen's Interoperability class to convert the LangChain tool
# This makes the Tavily tool usable by AutoGen agents

# Create the interoperability converter
interop = Interoperability()

# Convert the LangChain tool to AutoGen format
# The 'type' parameter specifies the source framework
ag2_tool = interop.convert_tool(
    tool=tavily_tool,    # The LangChain tool to convert
    type="langchain"     # Specify it's a LangChain tool
)

print("✅ Tool converted to AutoGen format!")

In [ ]:
# ============================================================================
# STEP 6C: REGISTER THE TOOL WITH AGENTS
# ============================================================================
# Tools must be registered with:
#   1. The LLM agent (so it knows about the tool and can call it)
#   2. The executor agent (so it can actually run the tool)

# Register with news_agent so its LLM knows about the tool
# This adds the tool's schema to the agent's available functions
ag2_tool.register_for_llm(news_agent)

# Register with user_proxy so it can execute the tool
# The user_proxy will actually call the Tavily API
ag2_tool.register_for_execution(user_proxy)

print("✅ Tavily tool registered!")
print("   - news_agent can now request Tavily searches")
print("   - user_proxy can execute the searches")

In [ ]:
# ┌─────────────────────────────────────────────────────────────────────┐
# │  DECORATOR PATTERN (Custom Functions)                               │
# │  ─────────────────────────────────────                               │
# │                                                                      │
# │    @agent.register_for_llm(...)     ←── Decorate at definition      │
# │    @proxy.register_for_execution()                                   │
# │    def my_function():               ←── You write this function     │
# │        ...                                                           │
# └─────────────────────────────────────────────────────────────────────┘
# ┌─────────────────────────────────────────────────────────────────────┐
# │  METHOD PATTERN (Existing Tools)                                     │
# │  ───────────────────────────────                                     │
# │                                                                      │
# │    langchain_tool = TavilySearch()  ←── Tool already exists         │
# │    ag2_tool = interop.convert_tool(langchain_tool)                   │
# │    ag2_tool.register_for_llm(agent) ←── Register after creation     │
# │    ag2_tool.register_for_execution(proxy)                            │
# └─────────────────────────────────────────────────────────────────────┘

---

## Step 7: Create a Custom Web Scraping Tool

Now we'll create our own custom tool to scrape Analytics Vidhya's blog archive.

### Tool Registration Pattern

AutoGen uses decorators to register functions as tools:

```python
@agent.register_for_llm(description="Tool description for the LLM")
@executor.register_for_execution()
def my_custom_tool():
    # Tool implementation
    return results
```

### How Web Scraping Works

```
┌──────────────────┐    ┌──────────────────┐    ┌──────────────────┐
│  requests.get() │───▶│  BeautifulSoup   │───▶│  Extracted Data  │
│   (Fetch HTML)  │    │  (Parse HTML)    │    │   (Structured)   │
└──────────────────┘    └──────────────────┘    └──────────────────┘
```

In [ ]:
# ============================================================================
# STEP 7: CREATE CUSTOM ANALYTICS VIDHYA SCRAPING TOOL
# ============================================================================
# This custom tool scrapes the Analytics Vidhya blog archive
# It extracts article headings, descriptions, and URLs

# DECORATOR 1: Register with the LLM agent
# The description helps the LLM understand when to use this tool
@av_blog_agent.register_for_llm(
    description="Searches Analytics Vidhya blog to get the results for a given query"
)
# DECORATOR 2: Register with the executor agent
# This allows the user_proxy to actually execute the function
@user_proxy.register_for_execution()
def analytics_vidhya_search():
    """
    Extracts blog headings and descriptions from Analytics Vidhya blog archive.
    
    This function:
    1. Sends an HTTP GET request to the Analytics Vidhya blog archive
    2. Parses the HTML response using BeautifulSoup
    3. Extracts article headings, URLs, and descriptions
    4. Returns a list of dictionaries with the extracted data
    
    Returns:
        list: A list of dictionaries containing:
            - 'heading': Article title
            - 'URL': Link to the full article
            - 'description': Brief description of the article
    """
    
    # Step 1: Fetch the blog archive page
    response = requests.get("https://www.analyticsvidhya.com/blog-archive/")
    
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Step 3: Find all blog entry elements
    # Each blog entry is in an <li> with class 'row list-items'
    list_items = soup.find_all('li', class_='row list-items')
    
    results = []
    
    # Step 4: Extract data from each blog entry
    for item in list_items:
        # Extract the article heading from <h3> tag
        heading = item.find('h3', class_='fs-24 text-light-primary')
        heading_text = heading.text.strip() if heading else "No Heading"
        
        # Extract the URL from the parent <a> tag
        url = heading.find_parent('a')['href'] if heading else "No Link"
        
        # Extract the description from <p> tag
        description = item.find('p', class_='fs-16 text-dark-tertiary')
        description_text = description.text.strip() if description else "No Description"
        
        # Append the extracted data to results list
        results.append({
            'heading': heading_text,
            'URL': url,
            'description': description_text,
        })
    
    return results

print("✅ Analytics Vidhya Search Tool created and registered!")

### 📝 Understanding the Custom Tool

Let's break down what the `analytics_vidhya_search()` function does:

| Step | Action | Code |
|------|--------|------|
| 1 | **HTTP Request** | `requests.get(url)` fetches the webpage |
| 2 | **Parse HTML** | `BeautifulSoup(html, 'html.parser')` creates a parseable structure |
| 3 | **Find Elements** | `soup.find_all('li', class_='...')` locates all blog entries |
| 4 | **Extract Data** | Use `.find()` and `.text` to get specific content |

### The Decorator Pattern

```python
@av_blog_agent.register_for_llm(description="...")
@user_proxy.register_for_execution()
def analytics_vidhya_search():
    ...
```

- **`@register_for_llm`**: Tells the LLM about this tool (makes it available for the agent to call)
- **`@register_for_execution`**: Allows the executor to run the actual function

---

## Step 8: Create the Email Agent

The Email Agent is our final agent in the pipeline. It:
- Receives all the gathered content from previous chats
- Formats it into a professional newsletter
- Doesn't need any external tools (just LLM capabilities)

In [ ]:
# ============================================================================
# STEP 8: CREATE THE EMAIL AGENT
# ============================================================================
# The email agent formats all gathered content into a newsletter
# It doesn't need tools - just good writing capabilities from the LLM

email_agent = ConversableAgent(
    name="email_agent",
    system_message="""You are an expert newsletter writer. Your job is to:
    1. Take the provided articles and news content
    2. Organize it into a well-structured, engaging newsletter
    3. Include all relevant URLs for readers to explore further
    4. Use professional but friendly tone suitable for a corporate L&D newsletter
    
    Return 'TERMINATE' when the newsletter is complete.""",
    llm_config=config_list
)

print("✅ Email Agent created!")

---

## Step 9: Define the Task Messages

Each chat in our sequential pipeline needs a task message. The messages define:
1. What the agent should do
2. What output is expected

### Sequential Chat Flow

```
Message 1 → av_blog_agent → Find AI agent articles from Analytics Vidhya
         ↓
         [Summary passed as context]
         ↓
Message 2 → news_agent → Find related news using Tavily
         ↓
         [Summary passed as context]
         ↓
Message 3 → email_agent → Write the newsletter
```

In [ ]:
# ============================================================================
# STEP 9: DEFINE TASK MESSAGES FOR EACH CHAT
# ============================================================================
# These messages will be sent to each agent in the sequential pipeline
# Each message defines a specific task for that stage

messages = [
    # Message 1: For the AV Blog Agent
    # Task: Search Analytics Vidhya and find AI agent content
    "Find the content on AI agents and include the URL in the content",
    
    # Message 2: For the News Agent (receives context from Chat 1)
    # Task: Find related news and combine with previous content
    """Find the latest news about the topics mentioned in the given content. 
    Result should include both initial content and news results""",
    
    # Message 3: For the Email Agent (receives context from Chats 1 and 2)
    # Task: Compile everything into a newsletter
    "Write a newsletter email for the given list of articles and news."
]

print("✅ Task messages defined!")
for i, msg in enumerate(messages, 1):
    print(f"   Chat {i}: {msg[:50]}...")

---

## Step 10: Execute the Sequential Chat Pipeline

Now we'll run the entire pipeline using `autogen.initiate_chats()`. This function:

1. Executes each chat in sequence
2. Automatically passes summaries between chats as context
3. Returns results from all chats

### Chat Configuration Options

| Parameter | Description |
|-----------|-------------|
| `sender` | The agent initiating the conversation |
| `recipient` | The agent responding |
| `message` | The task/prompt for this chat |
| `clear_history` | Whether to clear previous chat history |
| `summary_method` | How to summarize (`"last_msg"` uses last message) |
| `max_turns` | Maximum conversation turns |

In [ ]:
# ============================================================================
# STEP 10: EXECUTE THE SEQUENTIAL CHAT PIPELINE
# ============================================================================
# initiate_chats() runs multiple chats in sequence
# Each chat's summary is passed as context to the next chat

chat_results = autogen.initiate_chats(
    [
        # ═══════════════════════════════════════════════════════════════
        # CHAT 1: User Proxy → AV Blog Agent
        # Purpose: Find AI agent content from Analytics Vidhya
        # ═══════════════════════════════════════════════════════════════
        {
            "sender": user_proxy,           # User initiates the chat
            "recipient": av_blog_agent,     # AV Blog agent responds
            "message": messages[0],          # The search task
            "clear_history": True,           # Start fresh
            "summary_method": "last_msg",    # Use last message as summary
        },
        
        # ═══════════════════════════════════════════════════════════════
        # CHAT 2: User Proxy → News Agent  
        # Purpose: Find related news using Tavily
        # Note: Receives Chat 1's summary as context automatically
        # ═══════════════════════════════════════════════════════════════
        {
            "sender": user_proxy,           # User initiates
            "recipient": news_agent,        # News agent responds
            "message": messages[1],          # The news search task
            "summary_method": "last_msg",    # Use last message as summary
            # Context from Chat 1 is automatically appended!
        },
        
        # ═══════════════════════════════════════════════════════════════
        # CHAT 3: News Agent → Email Agent
        # Purpose: Write the final newsletter
        # Note: Receives summaries from Chats 1 and 2 as context
        # ═══════════════════════════════════════════════════════════════
        {
            "sender": news_agent,           # News agent initiates
            "recipient": email_agent,       # Email agent responds
            "message": messages[2],          # The newsletter task
            "max_turns": 1                   # Single turn for email generation
            # Context from Chats 1 & 2 is automatically appended!
        },
    ]
)

print("\n" + "="*60)
print("✅ Sequential chat pipeline completed!")
print("="*60)

---

## Step 11: Display the Final Newsletter

The `chat_results` list contains the results from all three chats:
- `chat_results[0]` - Analytics Vidhya search results
- `chat_results[1]` - News search results
- `chat_results[2]` - The final newsletter (this is what we want!)

Each result object has a `.summary` attribute containing the output.

In [ ]:
# ============================================================================
# STEP 11: DISPLAY THE FINAL NEWSLETTER
# ============================================================================
# Extract and display the final newsletter from the third chat's summary
# We use IPython's Markdown display for nice rendering

print("📧 FINAL NEWSLETTER OUTPUT:")
print("=" * 60)

# Get the summary from the third chat (index 2)
# The summary contains the formatted newsletter
newsletter = chat_results[2].summary

# Display as formatted Markdown
display(Markdown(newsletter))

---

# 📚 Summary: What We Learned

## Key Concepts

### 1. Sequential Chats
- Chain multiple agent conversations together
- Each chat's summary automatically passes to the next as context
- Great for multi-step workflows

### 2. Tool Integration
- **LangChain Interoperability**: Convert tools from other frameworks using `Interoperability.convert_tool()`
- **Custom Tools**: Create functions and register them with decorators
- **Dual Registration**: Tools need both LLM registration (for calling) and execution registration (for running)

### 3. Agent Roles

| Agent | Role | Tools Used |
|-------|------|------------|
| User Proxy | Tool executor | N/A (executes others' tools) |
| AV Blog Agent | Content finder | Custom web scraper |
| News Agent | News searcher | Tavily Search |
| Email Agent | Content formatter | None (LLM only) |

## Next Steps

- Explore other LangChain tools to integrate
- Add more agents for additional processing steps
- Implement email sending functionality
- Add scheduling for automated newsletters

---

**🎉 Congratulations! You've built a complete multi-agent newsletter generation system!**